<a href="https://colab.research.google.com/github/JHY3066/MiChung/blob/main/%EC%A0%9C%EC%B6%9C%ED%8C%8C%EC%9D%BC/gmu_ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import grangercausalitytests
import warnings

warnings.filterwarnings("ignore")

print("🚀 0.4점 돌파를 위한 최종 합체 시작...")

# 1. 데이터 로드
# (파일명이 정확한지 확인해주세요)
file_base = 'common_pairs_best_values_1620.csv'  # 0.311점 기록한 파일 (우리의 Base)
file_source = 'v3_lightgbm_submission.csv'     # 값을 가져올 1등 파일
train_file = 'train.csv'

df_base = pd.read_csv(file_base)
df_source = pd.read_csv(file_source)
train = pd.read_csv(train_file)

# Pair 식별자 생성
df_base['pair'] = df_base['leading_item_id'] + '_' + df_base['following_item_id']
df_source['pair'] = df_source['leading_item_id'] + '_' + df_source['following_item_id']

# 2. 후보군 선정 (Base에는 없지만 1등 파일에는 있었던 '버려진 쌍'들)
base_pairs = set(df_base['pair'])
source_pairs = set(df_source[df_source['value'] > 0]['pair'])
candidates = list(source_pairs - base_pairs)

print(f"✅ 기존 베이스 쌍: {len(base_pairs)}개")
print(f"🔎 정밀 검증할 후보 쌍: {len(candidates)}개")

# 3. 데이터 피벗 (검증용)
train['date'] = pd.to_datetime(train['year'].astype(str) + '-' + train['month'].astype(str) + '-01')
pivot_df = train.pivot_table(index='date', columns='item_id', values='value', aggfunc='sum').fillna(0)

# ---------------------------------------------------------
# 전략 1: TLCC (Time Lagged Cross Correlation) - 시차 1~6개월 스캔
# ---------------------------------------------------------
tlcc_rescued = []
tlcc_threshold = 0.65 # 기준값

for pair in candidates:
    lead, follow = pair.split('_')
    if lead in pivot_df.columns and follow in pivot_df.columns:
        s_lead = pivot_df[lead]
        s_follow = pivot_df[follow]

        max_corr = 0
        # 1개월부터 6개월까지 밀어보며 확인
        for lag in range(1, 7):
            if len(s_lead) > lag:
                # Lead(t-lag) vs Follow(t)
                corr = np.corrcoef(s_lead.iloc[:-lag], s_follow.iloc[lag:])[0, 1]
                if corr > max_corr:
                    max_corr = corr

        if max_corr > tlcc_threshold:
            tlcc_rescued.append(pair)

print(f"💎 TLCC(시차분석)로 발굴한 쌍: {len(tlcc_rescued)}개")

# ---------------------------------------------------------
# 전략 2: Granger Causality Test - 인과관계 검증
# ---------------------------------------------------------
granger_rescued = []
# 시간 단축을 위해 TLCC에서 못 찾은 애들 위주로 보거나, 전체를 다 봐도 됨 (여기선 전체 봅니다)
for pair in candidates:
    lead, follow = pair.split('_')
    if lead in pivot_df.columns and follow in pivot_df.columns:
        data = pivot_df[[follow, lead]]
        try:
            # Lag 1~3에 대해 테스트
            gc_res = grangercausalitytests(data, maxlag=3, verbose=False)
            p_values = [gc_res[lag][0]['ssr_ftest'][1] for lag in range(1, 4)]
            min_p = min(p_values)

            if min_p < 0.05: # 유의수준 0.05 미만 (인과관계 있음)
                granger_rescued.append(pair)
        except:
            continue

print(f"🔥 Granger(인과관계)로 발굴한 쌍: {len(granger_rescued)}개")

# ---------------------------------------------------------
# 4. 최종 합체 (Base + TLCC + Granger)
# ---------------------------------------------------------
final_pairs = base_pairs.union(set(tlcc_rescued)).union(set(granger_rescued))

# 1등 파일에서 해당 쌍들의 데이터 가져오기
final_df = df_source[df_source['pair'].isin(final_pairs)].copy()

# 저장
output_name = 'submission_final_ensemble_04_aiming.csv'
final_df[['leading_item_id', 'following_item_id', 'value']].to_csv(output_name, index=False)

print("-" * 30)
print(f"🎉 최종 파일 생성 완료: {output_name}")
print(f"📊 총 제출 쌍 개수: {len(final_df)}개 (기존 {len(base_pairs)} + 추가 {len(final_df) - len(base_pairs)})")
print("이 파일로 0.4점 돌파에 도전하세요!")

🚀 0.4점 돌파를 위한 최종 합체 시작...
✅ 기존 베이스 쌍: 1620개
🔎 정밀 검증할 후보 쌍: 538개
💎 TLCC(시차분석)로 발굴한 쌍: 32개
🔥 Granger(인과관계)로 발굴한 쌍: 202개
------------------------------
🎉 최종 파일 생성 완료: submission_final_ensemble_04_aiming.csv
📊 총 제출 쌍 개수: 1836개 (기존 1620 + 추가 216)
이 파일로 0.4점 돌파에 도전하세요!


In [6]:
import pandas as pd
import numpy as np

# 1. 파일 불러오기 (파일명이 정확한지 확인해주세요)
# (1) 우리가 만든 1,836개짜리 타겟 파일
target_file = 'submission_final_ensemble_04_aiming.csv'

# (2) 앙상블에 사용할 3개 베이스 파일
file_lgbm = 'v3_lightgbm_submission.csv'
file_pearson = 'submission_single_filter_pearson_filtered_only_0.35.csv'
file_sub3 = 'submission 0.3.csv'

df_target = pd.read_csv(target_file)
df_lgbm = pd.read_csv(file_lgbm)
df_pearson = pd.read_csv(file_pearson)
df_sub3 = pd.read_csv(file_sub3)

print("파일 로드 완료!")

# 2. Pair 식별자 생성 (선행_후행)
for df in [df_target, df_lgbm, df_pearson, df_sub3]:
    df['pair'] = df['leading_item_id'] + '_' + df['following_item_id']

# 3. 값 매핑을 위한 인덱스 설정 (빠른 검색용)
val_lgbm = df_lgbm.set_index('pair')['value']
val_pearson = df_pearson.set_index('pair')['value']
val_sub3 = df_sub3.set_index('pair')['value']

# 4. 공통 쌍(Common Pairs) 식별
# 3개 파일 모두가 "공행성 있다(Value > 0)"고 판단한 교집합
pairs_lgbm = set(df_lgbm[df_lgbm['value'] > 0]['pair'])
pairs_pearson = set(df_pearson[df_pearson['value'] > 0]['pair'])
pairs_sub3 = set(df_sub3[df_sub3['value'] > 0]['pair'])

common_pairs = pairs_lgbm.intersection(pairs_pearson).intersection(pairs_sub3)

print(f"✅ 3개 모델 공통 쌍: {len(common_pairs)}개 (평균값 적용)")
print(f"⭐ 새로 추가된 히든 쌍: {len(df_target) - len(common_pairs)}개 (1등 모델 값 유지)")

# 5. 하이브리드 값 계산 & 정수 변환
new_values = []

for idx, row in df_target.iterrows():
    pair = row['pair']

    if pair in common_pairs:
        # [전략 A] 공통 쌍이면 -> 3개 모델의 평균값 사용 (NMAE 오차 최소화)
        # .get(pair, 0)은 혹시 값이 없을 경우 에러 방지용
        mean_val = (val_lgbm.get(pair, 0) + val_pearson.get(pair, 0) + val_sub3.get(pair, 0)) / 3
        new_values.append(mean_val)
    else:
        # [전략 B] 추가된 히든 쌍이면 -> 가장 성능 좋은 LightGBM 값 그대로 사용
        # (다른 모델들은 이 쌍을 0으로 예측했을 가능성이 높아 평균 내면 값이 망가짐)
        new_values.append(row['value'])

# 값 업데이트
df_target['value'] = new_values

# ★ 핵심: 예측값을 반올림하여 정수(int)로 변환 (대회 규칙 준수)
df_target['value'] = df_target['value'].round().astype(int)

# 6. 최종 파일 저장
output_name = 'submission_final_ensemble_04_aiming_mean_int.csv'
df_target[['leading_item_id', 'following_item_id', 'value']].to_csv(output_name, index=False)

print("-" * 30)
print(f"🎉 파일 생성 완료: {output_name}")
print(f"데이터 확인:\n{df_target.head()}")

파일 로드 완료!
✅ 3개 모델 공통 쌍: 1620개 (평균값 적용)
⭐ 새로 추가된 히든 쌍: 216개 (1등 모델 값 유지)
------------------------------
🎉 파일 생성 완료: submission_final_ensemble_04_aiming_mean_int.csv
데이터 확인:
  leading_item_id following_item_id    value               pair
0        AANGBULD          DDEXPPXU    16186  AANGBULD_DDEXPPXU
1        AANGBULD          DEWLVASR   321588  AANGBULD_DEWLVASR
2        AANGBULD          EVBVXETX  4971642  AANGBULD_EVBVXETX
3        AANGBULD          FITUEHWN    75752  AANGBULD_FITUEHWN
4        AANGBULD          FTSVTTSR   128126  AANGBULD_FTSVTTSR


In [7]:
import pandas as pd
import numpy as np

# 1. 파일 불러오기 (경로/파일명 확인해주세요)
# (1) 우리가 만든 0.311점짜리 베이스 파일 (1,620개)
file_common = 'common_pairs_ensemble_1620_clean.csv'
# (2) 후보를 가져올 1등 파일 (LightGBM)
file_lgbm = 'v3_lightgbm_submission.csv'
# (3) 상관계수 계산을 위한 학습 데이터
file_train = 'train.csv'

df_common = pd.read_csv(file_common)
df_lgbm = pd.read_csv(file_lgbm)
train = pd.read_csv(file_train)

print("파일 로드 완료!")

# 2. Pair 식별자 생성
df_common['pair'] = df_common['leading_item_id'] + '_' + df_common['following_item_id']
df_lgbm['pair'] = df_lgbm['leading_item_id'] + '_' + df_lgbm['following_item_id']

# 3. 추가할 후보군(Lost Pairs) 찾기
# LightGBM에는 있는데, 안전빵 파일(Common)에는 없는 것들
pairs_common = set(df_common['pair'])
pairs_lgbm = set(df_lgbm[df_lgbm['value'] > 0]['pair'])

lost_pairs = list(pairs_lgbm - pairs_common)
print(f"베이스 파일 쌍: {len(pairs_common)}개")
print(f"추가 검증할 후보 쌍: {len(lost_pairs)}개")

# 4. 후보군 점수 매기기 (상관계수 분석)
# 데이터 피벗 (날짜별 정렬)
train['date'] = pd.to_datetime(train['year'].astype(str) + '-' + train['month'].astype(str) + '-01')
pivot_df = train.pivot_table(index='date', columns='item_id', values='value', aggfunc='sum').fillna(0)

scored_candidates = []

for pair in lost_pairs:
    lead, follow = pair.split('_')

    # 데이터가 있는 경우에만 계산
    if lead in pivot_df.columns and follow in pivot_df.columns:
        s_lead = pivot_df[lead]
        s_follow = pivot_df[follow]

        # 최대 상관계수 찾기 (Lag 0 ~ 6개월)
        corrs = []
        # (1) 단순 상관계수 (Lag 0)
        corrs.append(s_lead.corr(s_follow))
        # (2) 시차 상관계수 (Lag 1~6)
        for lag in range(1, 7):
            if len(s_lead) > lag:
                c = np.corrcoef(s_lead.iloc[:-lag], s_follow.iloc[lag:])[0, 1]
                corrs.append(c)

        # 가장 높은 상관계수를 점수로 사용
        max_corr = max(corrs) if corrs else 0

        # LightGBM의 예측값 가져오기
        val = df_lgbm.loc[df_lgbm['pair'] == pair, 'value'].values[0]

        scored_candidates.append({
            'leading_item_id': lead,
            'following_item_id': follow,
            'value': val,
            'pair': pair,
            'score': max_corr
        })

# 5. 상위 378개 선정
df_candidates = pd.DataFrame(scored_candidates)
df_candidates = df_candidates.sort_values('score', ascending=False) # 점수 높은 순 정렬

top_378 = df_candidates.head(378)
print(f"선정된 378개 중 최소 상관계수: {top_378['score'].min():.4f}")

# 6. 파일 합치기 (1,620개 + 378개 = 1,998개)
# 컬럼 순서 맞추기
cols = ['leading_item_id', 'following_item_id', 'value']
df_final = pd.concat([df_common[cols], top_378[cols]])

# ★ 정수 반올림 (규칙 준수)
df_final['value'] = df_final['value'].round().astype(int)

# 7. 저장
output_name = 'submission_gap_closer_1998.csv'
df_final.to_csv(output_name, index=False)

print("-" * 30)
print(f"🎉 파일 생성 완료: {output_name}")
print(f"총 쌍 개수: {len(df_final)}개")

파일 로드 완료!
베이스 파일 쌍: 1620개
추가 검증할 후보 쌍: 538개
선정된 378개 중 최소 상관계수: 0.2938
------------------------------
🎉 파일 생성 완료: submission_gap_closer_1998.csv
총 쌍 개수: 1998개


In [11]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import spearmanr
from tqdm import tqdm
import warnings

# 경고 무시
warnings.filterwarnings("ignore")

# ======================================================================================
# 1. 데이터 로드 및 전처리
# ======================================================================================
print("🚀 [Step 1] 데이터 로드 및 전처리...")
try:
    train = pd.read_csv('train.csv')
except FileNotFoundError:
    print("❌ 'train.csv' 파일이 없습니다.")
    raise

# 날짜 변환
train['date'] = pd.to_datetime(train['year'].astype(str) + '-' + train['month'].astype(str) + '-01')

# 피벗 테이블 (Index: Date, Columns: Item_ID, Values: Value)
pivot_df = train.pivot_table(index='date', columns='item_id', values='value', aggfunc='sum').fillna(0)
# 로그 변환 (학습용)
pivot_log = np.log1p(pivot_df)

items = pivot_df.columns.tolist()
print(f"✅ 총 아이템: {len(items)}개")

# ======================================================================================
# 2. 공행성 쌍 발굴 (엄격한 필터링)
# ======================================================================================
print("\n🚀 [Step 2] 공행성 쌍 점수 산출 (Smart Cutoff)...")

pair_scores = []
# v3 파일 참고: 임계값 0.35 근처가 정답률이 높음
MIN_THRESHOLD = 0.30  # 최소 이 점수는 넘어야 함

for lead in tqdm(items, desc="Scanning Pairs"):
    for follow in items:
        if lead == follow: continue

        s_lead = pivot_log[lead]
        s_follow = pivot_log[follow]

        # 1) Pearson (선형)
        pearson = s_lead.corr(s_follow)

        # 2) Spearman (비선형)
        spearman, _ = spearmanr(s_lead, s_follow)

        # 3) TLCC (시차 1~6개월) - v3 로직 반영
        lag_corrs = []
        for lag in range(1, 7):
            if len(s_lead) > lag:
                c = np.corrcoef(s_lead.iloc[:-lag], s_follow.iloc[lag:])[0, 1]
                lag_corrs.append(c)
        max_lag = max(lag_corrs) if lag_corrs else 0

        # 종합 점수 (최댓값 기준 - 하나라도 확실하면 채택)
        # 평균보다는 Max가 '숨겨진 관계'를 찾는 데 유리함
        final_score = max(pearson, spearman, max_lag)

        if final_score >= MIN_THRESHOLD:
            pair_scores.append({
                'pair_id': f"{lead}_{follow}",
                'lead': lead,
                'follow': follow,
                'score': final_score
            })

df_scores = pd.DataFrame(pair_scores).sort_values('score', ascending=False)

# 여기서 너무 많이 뽑히면 상위 2200개로 자르고, 적으면 그대로 감
if len(df_scores) > 2200:
    top_pairs = df_scores.head(2200)
    print(f"⚠️ 기준 통과 쌍이 많아 상위 2,200개만 선택했습니다.")
else:
    top_pairs = df_scores
    print(f"✅ 기준({MIN_THRESHOLD}) 통과 쌍 모두 선택.")

selected_pairs_set = set(top_pairs['pair_id'])
print(f"💎 최종 선발: {len(selected_pairs_set)}개 (최소 점수: {top_pairs['score'].min():.4f})")

# ======================================================================================
# 3. 데이터셋 구축 (v3 Feature Engineering 반영)
# ======================================================================================
print("\n🚀 [Step 3] 학습 데이터셋 구축...")

train_data = []
test_data = []

lags = [1, 2, 3, 6]

for _, row in tqdm(top_pairs.iterrows(), total=len(top_pairs), desc="Building Dataset"):
    lead = row['lead']
    follow = row['follow']
    pair_id = row['pair_id']

    ts_lead = pivot_log[lead].values
    ts_follow = pivot_log[follow].values

    # --- Feature 생성 함수 ---
    def get_features(t_idx, _lead, _follow):
        feat = {}
        # Lag Features
        for l in lags:
            feat[f'lead_lag_{l}'] = _lead[t_idx-l]
            feat[f'follow_lag_{l}'] = _follow[t_idx-l]

        # Rolling Mean (이동평균)
        feat['lead_ma_3'] = np.mean(_lead[t_idx-3:t_idx])
        feat['follow_ma_3'] = np.mean(_follow[t_idx-3:t_idx])

        # Momentum (변화율) - v3 참고
        if t_idx >= 2:
            feat['lead_mom'] = _lead[t_idx-1] - _lead[t_idx-2]
            feat['follow_mom'] = _follow[t_idx-1] - _follow[t_idx-2]
        else:
            feat['lead_mom'] = 0
            feat['follow_mom'] = 0

        return feat

    # 1) 학습 데이터
    for t in range(max(lags), len(ts_lead)):
        sample = {'pair_id': pair_id, 'target': ts_follow[t]}
        sample.update(get_features(t, ts_lead, ts_follow))
        train_data.append(sample)

    # 2) 테스트 데이터 (2025-08)
    last_idx = len(ts_lead)
    test_sample = {'pair_id': pair_id}
    test_sample.update(get_features(last_idx, ts_lead, ts_follow))
    test_data.append(test_sample)

train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)

features = [c for c in train_df.columns if c not in ['pair_id', 'target']]
X = train_df[features]
y = train_df['target']
X_test = test_df[features]

# ======================================================================================
# 4. 모델링 (앙상블)
# ======================================================================================
print("\n🚀 [Step 4] 모델 학습 및 예측...")

# LightGBM (메인)
lgb_model = lgb.LGBMRegressor(n_estimators=1000, learning_rate=0.03, num_leaves=31, random_state=42, verbose=-1)
lgb_model.fit(X, y)
p_lgb = lgb_model.predict(X_test)

# XGBoost
xgb_model = xgb.XGBRegressor(n_estimators=1000, learning_rate=0.03, max_depth=6, random_state=42, n_jobs=-1)
xgb_model.fit(X, y)
p_xgb = xgb_model.predict(X_test)

# Random Forest
rf_model = RandomForestRegressor(n_estimators=300, max_depth=12, random_state=42, n_jobs=-1)
rf_model.fit(X, y)
p_rf = rf_model.predict(X_test)

# 앙상블
p_log_avg = (p_lgb + p_xgb + p_rf) / 3
final_preds = np.expm1(p_log_avg)

# ======================================================================================
# 5. 후처리 및 저장 (Post-processing)
# ======================================================================================
print("\n🚀 [Step 5] 후처리 및 저장...")

# 과거 최대값 기반 Cap 적용 (gmu_1.ipynb 아이디어)
# 예측값이 과거 최대값의 1.5배를 넘지 않도록 제한 (폭발 방지 안전장치)
final_pred_map = {}

for i, pair in enumerate(test_df['pair_id']):
    lead, follow = pair.split('_')
    pred_val = max(0, final_preds[i])

    # Safety Cap
    hist_vals = pivot_df[follow].values
    max_val = np.max(hist_vals)
    if max_val > 0:
        # 과거 최대값보다 너무 크면 제한 (보수적 접근)
        cap_val = max_val * 1.5
        if pred_val > cap_val:
            pred_val = cap_val

    final_pred_map[pair] = pred_val

# 최종 파일 생성
submission_rows = []
for lead in items:
    for follow in items:
        if lead == follow: continue
        pair_key = f"{lead}_{follow}"

        val = 0
        if pair_key in final_pred_map:
            val = final_pred_map[pair_key]

        submission_rows.append({
            'leading_item_id': lead,
            'following_item_id': follow,
            'value': int(round(val))
        })

final_submission = pd.DataFrame(submission_rows)
output_name = 'final_submission_smart_cutoff.csv'
final_submission.to_csv(output_name, index=False)

print("-" * 50)
print(f"🎉 완료! 파일명: {output_name}")
print(f"예측된 쌍 개수: {len(final_pred_map)}")
print("-" * 50)

🚀 [Step 1] 데이터 로드 및 전처리...
✅ 총 아이템: 100개

🚀 [Step 2] 공행성 쌍 점수 산출 (Smart Cutoff)...


Scanning Pairs: 100%|██████████| 100/100 [00:34<00:00,  2.93it/s]


⚠️ 기준 통과 쌍이 많아 상위 2,200개만 선택했습니다.
💎 최종 선발: 2200개 (최소 점수: 0.3141)

🚀 [Step 3] 학습 데이터셋 구축...


Building Dataset: 100%|██████████| 2200/2200 [00:01<00:00, 1127.47it/s]



🚀 [Step 4] 모델 학습 및 예측...

🚀 [Step 5] 후처리 및 저장...
--------------------------------------------------
🎉 완료! 파일명: final_submission_smart_cutoff.csv
예측된 쌍 개수: 2200
--------------------------------------------------
